In [5]:
import os
import pandas as pd
import pickle

In [40]:
#!pip install plotly
pd.options.plotting.backend = "plotly"

In [2]:
dir_run03 = '../out/20.0819 modcompr/'

In [98]:
model_dirs = {'elasticNet': '../out/20.0819 modcompr/reg_elasticNet_infer/',
              'lm': '../out/20.0819 modcompr/reg_lm_infer/', 
              'rf': '../out/20.0819 modcompr/reg_rf_infer/', 
              'rf_boruta': '../out/20.0216 feat/reg_rf_boruta'
             }

### model results

In [99]:
scores = {}
for model, model_dir in model_dirs.items():
    res = pd.read_csv(os.path.join(model_dir, 'model_results.csv'))
    scores.update({model: res.loc[res.model=='all','score_test'].values})

scores = pd.DataFrame.from_dict(scores)

In [100]:
scores.mean(axis=0)

elasticNet    0.137925
lm            0.030242
rf            0.086768
rf_boruta     0.086768
dtype: float64

In [101]:
scores.boxplot()

In [109]:
res = pd.read_csv(os.path.join(model_dirs['elasticNet'], 'model_results.csv'))
res.loc[res.model=='all',['score_train', 'score_test']].describe()

,score_train,score_test
count,583.000000,583.000000
mean,0.222645,0.137925
std,0.148122,0.187106
min,0.000000,-2.455304
25%,0.110866,0.036640
50%,0.186384,0.106759
75%,0.311160,0.221161
max,0.764792,0.761665


In [111]:
res = pd.read_csv(os.path.join(model_dirs['lm'], 'model_results.csv'))
res.loc[res.model=='all',['score_train', 'score_test']].describe()

,score_train,score_test
count,583.0,583.000000
mean,1.0,0.030242
std,0.0,0.182642
min,1.0,-0.688894
25%,1.0,-0.076855
50%,1.0,0.046777
75%,1.0,0.149190
max,1.0,0.643561


Although the linear models look ok, but they overfitted quite substantially. Next we'll look at the results after feature selection, so to minimize overfitting on these linear models (the random forests don't overfit so we'll directly look at the test scores).

### anlyz

aggRes stats_score aggregates the scores, without excluding negative scores

feat stats_score aggregates the scores, and excludes negative scores

In [116]:
scores_fl = {}
scores_rd = {}
scores_rd10 = {}
for model, model_dir in model_dirs.items():
    res = pd.read_csv(os.path.join(model_dir, 'anlyz/stats_score_aggRes/stats_score.csv'), index_col=0)
    scores_fl.update({model: res.loc[res.index=='mean', 'full'].values})
    scores_rd.update({model: res.loc[res.index=='mean', 'reduced'].values})
    scores_rd10.update({model: res.loc[res.index=='mean', 'reduced10feat'].values})

scores_fl = pd.DataFrame.from_dict(scores_fl)
scores_rd = pd.DataFrame.from_dict(scores_rd)
scores_rd10 = pd.DataFrame.from_dict(scores_rd10)

In [124]:
df = pd.concat([scores_fl.mean(), 
           scores_rd.mean(),
           scores_rd10.mean()], axis=1)
df.columns = ['full', 'reduced', 'reduced_top10feat']
df

,full,reduced,reduced_top10feat
elasticNet,0.138418,0.138419,0.132926
lm,0.030242,0.037119,-0.071210
rf,0.086768,0.115086,0.285333
rf_boruta,0.086768,0.359706,0.389821


### anlyz_filtered

In [125]:
scores_fl = {}
scores_rd = {}
scores_rd10 = {}
for model, model_dir in model_dirs.items():
    res = pd.read_csv(os.path.join(model_dir, 'anlyz_filtered/stats_score_aggRes/stats_score.csv'), index_col=0)
    scores_fl.update({model: res.loc[res.index=='mean', 'full'].values})
    scores_rd.update({model: res.loc[res.index=='mean', 'reduced'].values})
    scores_rd10.update({model: res.loc[res.index=='mean', 'reduced10feat'].values})

scores_fl = pd.DataFrame.from_dict(scores_fl)
scores_rd = pd.DataFrame.from_dict(scores_rd)
scores_rd10 = pd.DataFrame.from_dict(scores_rd10)

In [126]:
df = pd.concat([scores_fl.mean(), 
           scores_rd.mean(),
           scores_rd10.mean()], axis=1)
df.columns = ['full', 'reduced', 'reduced_top10feat']
df

,full,reduced,reduced_top10feat
elasticNet,0.251047,0.251049,0.247983
lm,0.066277,0.074152,0.454841
rf,0.098717,0.132054,0.379765
rf_boruta,0.090035,0.382149,0.424561


Better to use anlyz, as anlyz_filtered applies filtering to the results, and the final list would be different among the different models